# Estadística para Ciencia de los Datos - Lección 7

Autor: Saúl Calderón, Juan Esquivel, Jorge Castro

# Remuestreo

Se le denomina remuestreo al proceso de tomar múltiples muestras sobre una misma población base. 
Los métodos estadísticos de remuestreo nos permiten  estimar parámetros poblacionales y **cuantificar la incertidumbre del estimado** (similar a los intervalos de confianza). Además se pueden usar para **evaluar y mejorar la exactitud de algunos modelos de aprendizaje automático**.


En general, los métodos de remuestreo son fáciles de usar y requieren poco conocimiento matemático en comparación a otros métodos de estadística inferencial. El remuestreo no se realiza para proveer un estimado sobre la distribución poblacional, sino más bien para proveer un estimado de la distribución muestral del estadístico de interés. A continuación veremos dos métodos de remuestreo: Bootstrap y Empaquetamiento (*bagging*).


## Bootstrap

Una manera efectiva y sencilla de estimar la distribución muestral de un estadístico, consiste en extraer muestras adicionales (bootstrap samples) con reemplazo de la misma muestra y recalcular el estadístico para cada remuestreo. A este procedimiento se le conoce como *Bootstrap* y no requiere necesariamente de ninguna suposición sobre la normalidad de los datos o el estadístico de interés.

El algoritmo de bootstrap para una muestra de tamaño $n$ con $R$ iteraciones se detalla a continuación:

1. Tomar una observación y guardarla con reemplazo (es decir, no se elimina la observación tomada del conjunto de datos).
2.  Repetir $n$ veces.
3. Calcular la media de los $n$ valores remuestreados.
4. Repetir los pasos del 1 al 3 $R$ veces.
5. Usar los $R$ resultados para:
  - Calcular su desviación estándar (que estima la desviación estándar de la media muestral $\sigma_{\overline{x}}$).
  - Producir un histograma o *boxplot*.
  - Encontrar un intervalo de confianza.

Entre más iteraciones $R$ se ejecuten más preciso será el estimado de $\sigma_{\overline{x}}$ y el intervalo de confianza. El resultado del algoritmo de *Bootstrap* es un conjunto de estadísticas muestrales o parámetros estimados del modelo que se pueden examinar para ver que tan variables son. En la práctica un modelo se puede ejecutar sobre los datos producidos con el Bootstrap para estimar la estabilidad de los parámetros del modelo o para mejorar su poder predictivo.

Es importante entender que la técnica de *bootstrap* no es una compensación para tamaños de muestra pequeños, tampoco crea datos nuevos ni rellena hoyos en un conjunto de datos existente. Tan solo informa sobre como una gran cantidad de muestras adicionales se comportarían si se toman de una población como la de nuestra muestra original. Una desventaja del bootstrap es que puede ser computacionalmente costoso.

Finalmente, sobre el Bootstrap cabe resaltar que:

1. Permite evaluar la variabilidad de un estadístico muestral.
2. Permite estimar las distribuciones muestrales de estadísticos para los cuales no se ha desarrollado una aproximación matemática.
3. Cuando se aplica modelos predictivos, la agregación de múltiples predicciones muestrales basadas en bootstrap (bagging) suele ser superior al uso de un único modelo

## Empaquetado o bagging

*Bagging* se refiere a un meta-algoritmo de aprendizaje por ensambles
o *consejos*, diseñado para mejorar la estabilidad de las predicciones en modelos de regresión o clasificación.

Sea un conjunto de observaciones de entrenamiento:

\begin{equation}
X=\left\{ \overrightarrow{x}_{1},\overrightarrow{x}_{2},\ldots,\overrightarrow{x}_{k}\right\} ,
\end{equation}

 el empaquetamiento genera entonces $m$ nuevos conjuntos de datos $\left\{ X_{1},\ldots,X_{m}\right\} $, con cada conjunto de datos compuesto por $k$ observaciones, muestreadas del conjunto de datos original $X$ **con reemplazo**. Para cada conjunto de datos $X_{i}$ (conocido como **muestra bootstrap**) el método de empaquetado propone crear y entrenar un modelo $c_{i}\left(X_i\right)$. Finalmente, 
los resultados de cada modelo son **promediados (para el caso de la regresión)**
o utilizados para realizar una **votación (en el caso de la
clasificación)**.

El empaquetado es recomendado para casos donde los modelos presentan
una variabilidad alta en sus salidas, por lo que puede estabilizar
las predicciones, sin embargo, en casos donde la variabilidad de la
predicción en el modelo sea baja, el empaquetado puede degradar la
exactitud de las predicciones. 

# Ejemplo diabetes

(Tomado de https://www.kaggle.com/kumargh/pimaindiansdiabetescsv):

Predicción sobre desarrollo de diabetes en pacientes (o no) en los próximos 5 años. Descripción de los atributos extraída de kaggle:

 $x_0$:  Number of times pregnant.
 
 $x_1:$ Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
 
 $x_2:$ Diastolic blood pressure (mm Hg).
 
 $x_3:$ Triceps skinfold thickness (mm).
 
 $x_4:$ 2-Hour serum insulin (mu U/ml).
 
 $x_5:$ Body mass index (weight in kg/(height in m)^2).
 
 $x_6:$ Diabetes pedigree function.
 
 $x_7: $Age (years).

La columna número 8 representa el objetivo de predicción $t$ (desarrollo de diabetes o no). Por lo tanto, cada observación está dado por $\vec{x} \in \mathbb{R}^8$ y el problema a abordar es una clasificación binaria (desarrollar diabetes o no).

El conjunto de datos tiene $m=768$ observaciones o muestras.

Un modelo lineal base da una certeza de 65% de predicción, mientras que los mejores resultados rondan 77%.

In [9]:
# from google.colab import files
# uploaded = files.upload()



In [6]:
from pandas import read_csv
import numpy as np
import io

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Perceptron

#read the dataset from csv
dataset = read_csv('pima-indians-diabetes.csv', header=None)

#print descriptive stats
print(dataset.describe())

# print the first 20 rows of data
print(dataset.head(20))

print("number of zeros per column")
print((dataset == 0).astype(int).sum())


                0           1           2           3           4           5  \
count  768.000000  768.000000  768.000000  768.000000  768.000000  768.000000   
mean     3.845052  120.894531   69.105469   20.536458   79.799479   31.992578   
std      3.369578   31.972618   19.355807   15.952218  115.244002    7.884160   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000   99.000000   62.000000    0.000000    0.000000   27.300000   
50%      3.000000  117.000000   72.000000   23.000000   30.500000   32.000000   
75%      6.000000  140.250000   80.000000   32.000000  127.250000   36.600000   
max     17.000000  199.000000  122.000000   99.000000  846.000000   67.100000   

                6           7           8  
count  768.000000  768.000000  768.000000  
mean     0.471876   33.240885    0.348958  
std      0.331329   11.760232    0.476951  
min      0.078000   21.000000    0.000000  
25%      0.243750   24.000000    0.000000  
50%   

## Marcar datos nulos

En algunas columnas como "Triceps skinfold thickness" o "diastolic pressure", los valores de cero no tienen sentido. 
Una posible estrategia para trabajar con estos valores inválidos o faltantes consiste en eliminar todas las observaciones (filas) que contengan al menos una columna con algún valor faltante.


Para las columnas del 1-5 los valores 0 primero serán reemplazados por `NaN` usando la función `replace`. Luego se borran todas las filas con valores `NaN`, lo cuál incrementa el riesgo de *over fitting*. (ejercicio posterior: usar al menos 2 técnicas más de imputación y aplicar la técnica de bagging)

In [7]:
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0, np.NaN)
print("Number of null observations per column: ")
print(dataset.isnull().sum())

#we can eliminate the observations or samples with nans
dataset = dataset.dropna()
print("Number of null observations per column after NaN entry deletion: ")
print(dataset.isnull().sum())

#print descriptive stats
print(dataset.describe())

Number of null observations per column: 
0      0
1      5
2     35
3    227
4    374
5     11
6      0
7      0
8      0
dtype: int64
Number of null observations per column after NaN entry deletion: 
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64
                0           1           2           3           4           5  \
count  392.000000  392.000000  392.000000  392.000000  392.000000  392.000000   
mean     3.301020  122.627551   70.663265   29.145408  156.056122   33.086224   
std      3.211424   30.860781   12.496092   10.516424  118.841690    7.027659   
min      0.000000   56.000000   24.000000    7.000000   14.000000   18.200000   
25%      1.000000   99.000000   62.000000   21.000000   76.750000   28.400000   
50%      2.000000  119.000000   70.000000   29.000000  125.500000   33.200000   
75%      5.000000  143.000000   78.000000   37.000000  190.000000   37.100000   
max     17.000000  198.000000  110.000000   63.000000  846.000000   67.1000

## Bagging / Ensemble Learning

Código base para aplicar *Bagging* (aprendizaje por ensambles
o *consejos*) para clasificación, con votación para la decisión final.

In [8]:
# split dataset into inputs and outputs
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

#Model test function
def testModel(dataset):
  values = dataset.values
  
  
  Xtraining = values[:292,0:8]
  Ttraining = values[:292,8]
  
  Xtest = values[292:,0:8]
  Ttest = values[292:,8]

  #use K-nearest Neighbors Classifier and two Decision Trees
  model1 = KNeighborsClassifier() 
  model2 = tree.DecisionTreeClassifier(criterion='entropy') 
  model3 = tree.DecisionTreeClassifier(criterion='gini')
  
  #fit models
  model1.fit(Xtraining, Ttraining)
  model2.fit(Xtraining, Ttraining) 
  model3.fit(Xtraining, Ttraining) 
  
  #make final predictions
  predictions1 = model1.predict(Xtest)
  predictions2 = model2.predict(Xtest)
  predictions3 = model3.predict(Xtest)
  
   
  finalPredictions = np.array([])
  
  for i in range(0, len(Ttest)):
    mode = stats.mode([predictions1[i], predictions2[i], predictions3[i]])    
    #print("Mode ", mode, " p1 ", predictions1[i],  " p2 ", predictions2[i], " p3 ", predictions3[i])   
    #Prediction of the ensemble
    finalPredictions = np.append(finalPredictions, mode[0]) 

  #Evaluation metric
  dist1 = accuracy_score(Ttest, predictions1) 
  dist2 = accuracy_score(Ttest, predictions2) 
  dist3 = accuracy_score(Ttest, predictions3) 
  distEnsemble = accuracy_score(Ttest, finalPredictions)
  
  print("D1 ", dist1, " D2 ", dist2, " D3 ", dist3)
  print("Ensemble ", distEnsemble)

  
#Function call  
testModel(dataset)

D1  0.76  D2  0.75  D3  0.73
Ensemble  0.79


## Ejercicio:

Haga las correcciones pertinentes para implementar correctamente la técnica de empaquetado o *bagging* de acuerdo al material visto en esta lección.

In [53]:
# split dataset into inputs and outputs
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Model test function
def testModel(features, labels):
    values = dataset.values
  
    # Se sustituye esta parte
    # Xtraining = values[:292,0:8]
    # Ttraining = values[:292,8]
    # Xtest = values[292:,0:8]
    # Ttest = values[292:,8]


    Xtraining, Xtest, Ttraining, Ttest = train_test_split(features, labels, test_size=0.30, random_state=42)

    #use K-nearest Neighbors Classifier and two Decision Trees
    model1 = KNeighborsClassifier() 
    model2 = tree.DecisionTreeClassifier(criterion='entropy') 
    model3 = tree.DecisionTreeClassifier(criterion='gini')

    #fit models
    model1.fit(Xtraining, Ttraining)
    model2.fit(Xtraining, Ttraining) 
    model3.fit(Xtraining, Ttraining) 

    #make final predictions
    predictions1 = model1.predict(Xtest)
    predictions2 = model2.predict(Xtest)
    predictions3 = model3.predict(Xtest)


    finalPredictions = np.array([])

    for i in range(0, len(Ttest)):
        mode = stats.mode([predictions1[i], predictions2[i], predictions3[i]])    
        #print("Mode ", mode, " p1 ", predictions1[i],  " p2 ", predictions2[i], " p3 ", predictions3[i])   
        #Prediction of the ensemble
        finalPredictions = np.append(finalPredictions, mode[0]) 

    #Evaluation metric
    dist1 = accuracy_score(Ttest, predictions1) 
    dist2 = accuracy_score(Ttest, predictions2) 
    dist3 = accuracy_score(Ttest, predictions3) 
    distEnsemble = accuracy_score(Ttest, finalPredictions)

    print("D1 ", dist1, " D2 ", dist2, " D3 ", dist3)
    print("Ensemble ", distEnsemble)
  
# Function call
features = dataset.values[:292,0:8]
print(features)
labels = dataset.iloc[:,[8]]
print(labels)
testModel(features, labels)

[[  1.     89.     66.    ...  28.1     0.167  21.   ]
 [  0.    137.     40.    ...  43.1     2.288  33.   ]
 [  3.     78.     50.    ...  31.      0.248  26.   ]
 ...
 [  1.     87.     68.    ...  37.6     0.401  24.   ]
 [  6.     99.     60.    ...  26.9     0.497  32.   ]
 [  2.     95.     54.    ...  26.1     0.748  22.   ]]
     8
3    0
4    1
6    1
8    1
13   1
..  ..
753  1
755  1
760  0
763  0
765  0

[392 rows x 1 columns]


ValueError: Found input variables with inconsistent numbers of samples: [292, 392]

In [54]:
features = dataset.values[:292,0:8]
print(features)


#labels = dataset.iloc[:,[8]]
#print(labels)


[[  1.     89.     66.    ...  28.1     0.167  21.   ]
 [  0.    137.     40.    ...  43.1     2.288  33.   ]
 [  3.     78.     50.    ...  31.      0.248  26.   ]
 ...
 [  1.     87.     68.    ...  37.6     0.401  24.   ]
 [  6.     99.     60.    ...  26.9     0.497  32.   ]
 [  2.     95.     54.    ...  26.1     0.748  22.   ]]


# Práctica para Exámen

1. Si se tienen 2 eventos $A$ y $B$ con probabilidades $p_1$ y $p_2$ respectivamente, ¿Cuál es la probabilidad máxima de ocurrencia de ambos eventos en términos de sus probabilidades individuales $p_1$ y $p_2$?   

2. Explique cuál es la diferencia entre la probabilidad conjunta de 2 variables aleatorias $X$ y $Y$ y la probabilidad condicional de las mismas. Matemáticamente, ¿Cómo se encuentran relacionadas estas probabilidades?

3. El 40% de las ventas en una tienda de ropa corresponden a artículos con descuento. Si los clientes devuelven el 15% de los artículos que compran con descuento y el 6% de los artículos que compran sin descuento ¿Cuál es el porcentaje global de artículos devueltos? Además, ¿Cuál es la probabilidad de que un cliente haya comprado un artículo con descuento dado que no fue devuelto?

4. ¿Cómo se calcula el rango intercuartil en los diagramas de cajas y que permite determinar?

5. Mediante un muestreo aleatorio se han recabado los siguientes tiempos de realización de exámenes del curso de estadística:

$90.5 \quad 187.2 \quad 54.7 \quad 127.4 \quad 182.1 \quad 260.5 \quad 140.3 \quad 209.2 \quad 277.9 \quad 10.7$

Obtenga la estimación de máxima verosimilitud del tiempo medio de realización de la prueba y su desviación estándar.

6. ¿Cuál es la diferencia entre un estimador puntual y un intervalo de confianza?¿Qué significa el nivel de confianza y como se expresa matemáticamente?

7. Suponga que la probabilidad de mal funcionamiento de un motor durante cualquier periodo de una hora es p = 0.02. Encuentre la probabilidad de que un motor determinado funcione bien 
al menos tres horas.

8. ¿Cuál es la diferencia entre la distribución t-student y la distribución Z? Según el contexto del curso ¿En qué casos conviene usar una o la otra?

9. El vicepresidente de ventas de una gran compañía afirma que los vendedores están promediando no más de 15 contactos de venta por semana. Como prueba de su afirmación, aleatoriamente se seleccionan $n=16$ vendedores y se registra el número de contactos hechos por cada uno para una sola semana seleccionada al azar. La media y varianza de las 16 mediciones fueron 17 y 9, respectivamente. ¿La evidencia contradice lo dicho por el vicepresidente? Use una prueba con nivel de significancia $\alpha=0.05$.

In [ ]:
from scipy.stats import norm, t
import numpy as np


In [5]:
import pandas as pd

url = "https://www.kaggle.com/kumargh/pimaindiansdiabetescsv"
df = pd.read_csv(url)
df

ParserError: Error tokenizing data. C error: Expected 1 fields in line 7, saw 2
